In [30]:
using PCT, Profile, PProf

In [25]:
f, ctx = @pct begin
    @space H begin
        type=(I, I)->C
        symmetries=(((2,1), :conj),)
    end
    
    @space V begin
        type=(I,)->C
    end

    @space T begin
        type=(I,I,I,I) -> C
        symmetries=(((2,1,4,3), :conj), ((3,4,1,2), :id))
    end
                
    @space U begin
        type=(I,I)->C
    end
                    
    (A::H, J::T) -> _
end

(Map(MapType(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), UndeterminedPCTType(), Dict{Any, Any}()), PCTVector(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), Var{MapType}[Var{MapType}(MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), :A), Var{MapType}(MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id)))), :J)]), Var{UndeterminedPCTTyp

In [31]:
g = @pct f ctx pullback((x::V) -> sum((i, j), x(i)' * A(i, j) * x(j)))

(A, J) -> 
    𝒫((x) -> 
        ∑((i), ∑((j), (x(i)'⋅A(i, j)⋅x(j)))))

In [32]:
p = fc(eval_all(reduce_pullback(g)))

(x, _K) -> 
    (i_0) -> 
        ∑((i), ∑((j), (δ(i_0, i, (A(i, j)⋅x(j)⋅_K'))+δ(i_0, j, (A(i, j)'⋅x(i)⋅_K)))))

In [33]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_0) -> 
    ∑((i), ∑((j), (δ(i_0, i, (A(i, j)⋅x(j)))+δ(i_0, j, (A(i, j)'⋅x(i))))))

In [34]:
simplify(q) |> first

  0.004242 seconds (4.24 k allocations: 187.352 KiB)
1 symmetry
  0.005613 seconds (12.85 k allocations: 557.227 KiB)
2 symmetry
  0.003038 seconds (8.72 k allocations: 382.453 KiB)
3 symmetry
  0.001077 seconds (8.21 k allocations: 363.953 KiB)
4 sum_dist
  0.002712 seconds (11.83 k allocations: 530.094 KiB)
5 symmetry
  0.000830 seconds (5.42 k allocations: 241.938 KiB)
6 symmetry
  0.001691 seconds (13.72 k allocations: 606.328 KiB)
7 symmetry
  0.001200 seconds (10.04 k allocations: 447.188 KiB)
8 contract_delta
directed!
  0.000235 seconds (1.67 k allocations: 76.594 KiB)
9 symmetry
  0.000400 seconds (2.10 k allocations: 96.078 KiB)
10 symmetry
  0.000252 seconds (2.04 k allocations: 92.594 KiB)
11 symmetry
  0.000244 seconds (1.77 k allocations: 80.875 KiB)
12 sum_dist
  0.002247 seconds (13.52 k allocations: 598.008 KiB)
13 symmetry
  0.002565 seconds (21.00 k allocations: 909.719 KiB)
14 sum_ex
  0.004663 seconds (43.14 k allocations: 1.829 MiB)
15 symmetry
  0.003203 seconds 

(i_0) -> 
    (∑((j), (A(i_0, j)⋅x(j)))⋅2)

In [7]:
g = @pct f ctx pzullback((C::U) -> sum((i,j,p,q,r,s), C(p,q)' * C(q,i) * C(r,j)' * C(s,j) * J(p,q,r,s)))

(A, J) -> 
    𝒫((C) -> 
        ∑((i), ∑((j), ∑((p), ∑((q), ∑((r), ∑((s), (C(p, q)'⋅C(q, i)⋅C(r, j)'⋅C(s, j)⋅J(p, q, r, s)))))))))

In [8]:
p = fc(eval_all(reduce_pullback(g)))

(C, _K) -> 
    (i_0, i_1) -> 
        ∑((i), ∑((j), ∑((p), ∑((q), ∑((r), ∑((s), (δ(i_1, q, δ(i_0, p, (C(q, i)⋅C(r, j)'⋅C(s, j)⋅J(p, q, r, s)⋅_K')))+δ(i_1, i, δ(i_0, q, (C(r, j)⋅C(s, j)'⋅J(p, q, r, s)'⋅C(p, q)⋅_K)))+δ(i_1, j, δ(i_0, r, (C(s, j)⋅J(p, q, r, s)⋅C(q, i)⋅C(p, q)'⋅_K')))+δ(i_1, j, δ(i_0, s, (J(p, q, r, s)'⋅C(r, j)⋅C(q, i)'⋅C(p, q)⋅_K))))))))))

In [9]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_0, i_1) -> 
    ∑((i), ∑((j), ∑((p), ∑((q), ∑((r), ∑((s), (δ(i_1, q, δ(i_0, p, (C(q, i)⋅C(r, j)'⋅C(s, j)⋅J(p, q, r, s))))+δ(i_1, i, δ(i_0, q, (C(r, j)⋅C(s, j)'⋅J(p, q, r, s)'⋅C(p, q))))+δ(i_1, j, δ(i_0, r, (C(s, j)⋅J(p, q, r, s)⋅C(q, i)⋅C(p, q)')))+δ(i_1, j, δ(i_0, s, (J(p, q, r, s)'⋅C(r, j)⋅C(q, i)'⋅C(p, q)))))))))))

In [12]:
simplify(q) |> first

  0.020201 seconds (199.20 k allocations: 9.167 MiB)
1 symmetry
  0.038427 seconds (301.51 k allocations: 13.790 MiB, 22.14% gc time)
2 symmetry
  0.051698 seconds (401.10 k allocations: 18.330 MiB)
3 symmetry
  0.046241 seconds (409.43 k allocations: 18.752 MiB)
4 delta_ex
  0.064733 seconds (497.59 k allocations: 22.870 MiB, 17.02% gc time)
5 symmetry
  0.054803 seconds (497.35 k allocations: 22.764 MiB)
6 symmetry
  0.055608 seconds (601.43 k allocations: 27.483 MiB)
7 symmetry
  0.068618 seconds (596.41 k allocations: 27.371 MiB, 13.06% gc time)
8 symmetry
  0.058041 seconds (600.36 k allocations: 27.612 MiB)
9 symmetry
  0.065629 seconds (602.71 k allocations: 27.635 MiB, 10.07% gc time)
10 symmetry
  0.061613 seconds (611.06 k allocations: 28.008 MiB)
11 symmetry
  0.067380 seconds (616.80 k allocations: 28.357 MiB, 9.66% gc time)
12 delta_ex
  0.065895 seconds (708.80 k allocations: 32.679 MiB)
13 symmetry
  0.071957 seconds (713.59 k allocations: 32.779 MiB, 8.41% gc time)
14 s

  0.132521 seconds (1.29 M allocations: 58.981 MiB, 5.41% gc time)
109 symmetry
  0.127543 seconds (1.29 M allocations: 59.127 MiB, 5.17% gc time)
110 symmetry
  0.126294 seconds (1.31 M allocations: 59.714 MiB, 5.67% gc time)
111 symmetry
  0.126268 seconds (1.29 M allocations: 59.270 MiB, 5.74% gc time)
112 symmetry
  0.125101 seconds (1.29 M allocations: 59.033 MiB, 5.11% gc time)
113 symmetry
  0.127786 seconds (1.30 M allocations: 59.558 MiB, 5.22% gc time)
114 symmetry
  0.131315 seconds (1.30 M allocations: 59.150 MiB, 5.01% gc time)
115 symmetry
  0.123891 seconds (1.28 M allocations: 58.874 MiB, 5.01% gc time)
116 delta_ex
  0.124238 seconds (1.30 M allocations: 59.513 MiB, 5.01% gc time)
117 symmetry
  0.125292 seconds (1.30 M allocations: 59.382 MiB, 4.96% gc time)
118 symmetry
  0.131787 seconds (1.31 M allocations: 59.674 MiB, 4.95% gc time)
119 symmetry
  0.134583 seconds (1.30 M allocations: 59.443 MiB, 4.96% gc time)
120 symmetry
  0.133811 seconds (1.30 M allocations: 

  0.156843 seconds (1.51 M allocations: 69.534 MiB, 4.56% gc time)
212 delta_ex
  0.154020 seconds (1.52 M allocations: 69.773 MiB, 4.65% gc time)
213 symmetry
  0.155683 seconds (1.53 M allocations: 69.842 MiB, 4.37% gc time)
214 symmetry
  0.162026 seconds (1.52 M allocations: 69.702 MiB, 8.40% gc time)
215 symmetry
  0.153028 seconds (1.50 M allocations: 69.193 MiB, 4.38% gc time)
216 symmetry
  0.156061 seconds (1.51 M allocations: 69.204 MiB, 4.36% gc time)
217 symmetry
  0.158777 seconds (1.52 M allocations: 69.600 MiB, 4.33% gc time)
218 symmetry
  0.155704 seconds (1.52 M allocations: 69.357 MiB, 4.43% gc time)
219 symmetry
  0.155444 seconds (1.51 M allocations: 69.224 MiB, 4.41% gc time)
220 delta_ex
  0.154281 seconds (1.52 M allocations: 69.507 MiB, 4.44% gc time)
221 symmetry
  0.155605 seconds (1.51 M allocations: 69.054 MiB, 4.36% gc time)
222 symmetry
  0.160606 seconds (1.51 M allocations: 69.059 MiB, 8.42% gc time)
223 symmetry
  0.154130 seconds (1.51 M allocations: 

  0.156900 seconds (1.52 M allocations: 69.497 MiB, 4.54% gc time)
315 symmetry
  0.156340 seconds (1.53 M allocations: 70.131 MiB, 4.49% gc time)
316 delta_ex
  0.147407 seconds (1.52 M allocations: 69.685 MiB, 4.50% gc time)
317 symmetry
  0.148147 seconds (1.52 M allocations: 69.475 MiB, 4.63% gc time)
318 symmetry
  0.156263 seconds (1.52 M allocations: 69.486 MiB, 4.45% gc time)
319 symmetry
  0.155935 seconds (1.53 M allocations: 70.036 MiB, 4.50% gc time)
320 symmetry
  0.163898 seconds (1.53 M allocations: 70.009 MiB, 8.60% gc time)
321 symmetry
  0.157108 seconds (1.53 M allocations: 69.486 MiB, 4.45% gc time)
322 symmetry
  0.155656 seconds (1.53 M allocations: 69.477 MiB, 4.50% gc time)
323 symmetry
  0.156322 seconds (1.54 M allocations: 70.133 MiB, 4.51% gc time)
324 delta_ex
  0.157385 seconds (1.54 M allocations: 70.195 MiB, 4.50% gc time)
325 symmetry
  0.148807 seconds (1.53 M allocations: 69.489 MiB, 4.48% gc time)
326 symmetry
  0.151431 seconds (1.53 M allocations: 

  0.157997 seconds (1.54 M allocations: 69.930 MiB, 4.56% gc time)
418 symmetry
  0.150181 seconds (1.54 M allocations: 69.990 MiB, 4.69% gc time)
419 symmetry
  0.154306 seconds (1.54 M allocations: 70.222 MiB, 4.45% gc time)
420 delta_ex
  0.155282 seconds (1.54 M allocations: 70.241 MiB, 4.95% gc time)
421 symmetry
  0.196300 seconds (1.54 M allocations: 69.746 MiB, 4.05% gc time)
422 symmetry
  0.158973 seconds (1.62 M allocations: 73.821 MiB, 4.59% gc time)
423 symmetry
  0.166773 seconds (1.63 M allocations: 74.409 MiB, 8.43% gc time)
424 symmetry
  0.167525 seconds (1.63 M allocations: 74.573 MiB, 4.49% gc time)
425 symmetry
  0.157925 seconds (1.63 M allocations: 74.244 MiB, 4.41% gc time)
426 symmetry
  0.157090 seconds (1.63 M allocations: 74.341 MiB, 4.45% gc time)
427 symmetry
  0.157565 seconds (1.63 M allocations: 74.651 MiB, 4.38% gc time)
428 delta_ex
  0.163035 seconds (1.63 M allocations: 74.529 MiB, 4.54% gc time)
429 symmetry
  0.166590 seconds (1.63 M allocations: 

  0.157832 seconds (1.63 M allocations: 75.005 MiB, 4.48% gc time)
521 symmetry
  0.158710 seconds (1.63 M allocations: 74.636 MiB, 4.44% gc time)
522 symmetry
  0.157960 seconds (1.63 M allocations: 74.807 MiB, 4.47% gc time)
523 symmetry
  0.159906 seconds (1.63 M allocations: 74.973 MiB, 4.67% gc time)
524 delta_ex
  0.169523 seconds (1.63 M allocations: 74.876 MiB, 8.76% gc time)
525 symmetry
  0.158368 seconds (1.63 M allocations: 74.627 MiB, 4.45% gc time)
526 symmetry
  0.157124 seconds (1.62 M allocations: 74.512 MiB, 4.49% gc time)
527 symmetry
  0.156852 seconds (1.62 M allocations: 74.609 MiB, 4.49% gc time)
528 symmetry
  0.156974 seconds (1.62 M allocations: 74.514 MiB, 4.48% gc time)
529 symmetry
  0.166786 seconds (1.62 M allocations: 74.314 MiB, 4.44% gc time)
530 symmetry
  0.167011 seconds (1.63 M allocations: 74.646 MiB, 8.64% gc time)
531 symmetry
  0.167795 seconds (1.62 M allocations: 74.698 MiB, 4.41% gc time)
532 delta_ex
  0.157758 seconds (1.63 M allocations: 

LoadError: InterruptException:

In [8]:
Profile.clear()
@profile p==p
pprof()

"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599
